#  CAR BRAND CLASSIFICATION - TRANSFER LEARNING USING VGG16

In [33]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [34]:
# import the libraries as shown below
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [35]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/My Drive/IMAGES/Train'
valid_path = '/content/drive/My Drive/IMAGES/Test'

In [36]:
# Import the vgg16 library as shown below and add preprocessing layer to the front of vgg16
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [37]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [38]:
# useful for getting number of output classes
folders = glob('/content/drive/My Drive/IMAGES/Train/*')

In [39]:
# output layers
x = Flatten()(vgg16.output)

In [40]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [41]:
# view the structure of the model
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [42]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [43]:
# Use the Image Data Generator to import the images from the dataset

train_data = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_data = ImageDataGenerator(rescale = 1./255)

In [44]:
# Make sure to provide the same target size as initialied for the image size
training_set = train_data.flow_from_directory('/content/drive/My Drive/IMAGES/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 55 images belonging to 3 classes.


In [45]:
test_set = test_data.flow_from_directory('/content/drive/My Drive/IMAGES/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 58 images belonging to 3 classes.


In [46]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
2/2 [==============================] - 26s 24s/step - loss: 1.6415 - accuracy: 0.2930 - val_loss: 1.6456 - val_accuracy: 0.3793
Epoch 2/50
2/2 [==============================] - 1s 590ms/step - loss: 1.0890 - accuracy: 0.5424 - val_loss: 1.8226 - val_accuracy: 0.5690
Epoch 3/50
2/2 [==============================] - 1s 585ms/step - loss: 1.2857 - accuracy: 0.5008 - val_loss: 1.0886 - val_accuracy: 0.6379
Epoch 4/50
2/2 [==============================] - 1s 600ms/step - loss: 0.6064 - accuracy: 0.7383 - val_loss: 0.8005 - val_accuracy: 0.6034
Epoch 5/50
2/2 [==============================] - 1s 587ms/step - loss: 0.6190 - accuracy: 0.7123 - val_loss: 0.5487 - val_accuracy: 0.7759
Epoch 6/50
2/2 [==============================] - 1s 592ms/step - loss: 0.2165 - accuracy: 0.9549 - val_loss: 0.7454 - val_accuracy: 0.6897
Epoch 7/50
2/2 [==============================] - 1s 694ms/step - loss: 0.2132 - accuracy: 0.8983 - val_loss: 1.0055 - val_accuracy: 0.6552
Epoch 8/50
2/2 [=====

In [47]:
from tensorflow.keras.models import load_model
model.save('model.h5')

In [48]:
y_pred = model.predict(test_set)
y_pred

array([[1.37769321e-05, 9.99637127e-01, 3.49034730e-04],
       [6.52512885e-04, 9.99080062e-01, 2.67350086e-04],
       [1.65053178e-02, 1.18306652e-02, 9.71664071e-01],
       [9.22915861e-02, 4.68349725e-01, 4.39358741e-01],
       [1.69892386e-02, 7.69144058e-01, 2.13866696e-01],
       [1.20231335e-03, 9.89297509e-01, 9.50009469e-03],
       [4.22380865e-04, 9.39818680e-01, 5.97589612e-02],
       [3.27131338e-02, 5.54243207e-01, 4.13043737e-01],
       [4.39757807e-03, 9.82141018e-01, 1.34613970e-02],
       [5.42382069e-04, 9.99384046e-01, 7.35101203e-05],
       [3.89003806e-04, 9.97124732e-01, 2.48618168e-03],
       [8.43151271e-01, 1.21006526e-01, 3.58421504e-02],
       [1.58621818e-02, 7.26058543e-01, 2.58079290e-01],
       [1.45185255e-02, 9.82809365e-01, 2.67216517e-03],
       [3.54258297e-03, 8.22292328e-01, 1.74165055e-01],
       [1.20112172e-03, 9.97500122e-01, 1.29880838e-03],
       [9.72549460e-05, 9.99254167e-01, 6.48582412e-04],
       [6.88594058e-02, 2.22881

In [49]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 0, 2, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 0, 1, 0, 1, 1, 2, 1, 2, 1, 2, 2,
       1, 2, 1, 1, 2, 0, 1, 1, 1, 1, 0, 2, 1, 0])

In [50]:
model=load_model('model.h5')

## MODEL PREDICTION

In [75]:
imag = image.load_img('/content/drive/My Drive/IMAGES/Test/audi/26.jpg', target_size = (224, 224))
x = image.img_to_array(imag)
x=x/255
test_image = np.expand_dims(x, axis = 0)
result = model.predict(test_image)
result=np.argmax(result, axis=1)
if result==0:
 print("The Car is Audi")
elif result==1:
  print("The Car is Lamborghini")
else:
  print("The Car Is Mercedes")

The Car is Audi
